In [1]:
from astropy.table import Table
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
import fitsio
import desimodel.io
import desitarget.mtl
import desisim.quickcat
from astropy.io import fits
from astropy.table import Table, Column, vstack
import json
import shutil

from desitarget.targetmask import desi_mask, obsconditions
from collections import Counter
import subprocess

%matplotlib inline

In [2]:
zcat = Table.read("strategy_A/zcat/dark2_dark3_zcat.fits")

In [33]:
zcat_large = zcat[zcat['NUMOBS']>5]

In [27]:
targets = Table.read("targets/subset_dr8_mtl_dark_gray_NGC.fits")

In [29]:
only_qso_not_elg = ((targets['DESI_TARGET']&desi_mask.QSO)!=0)&(((targets['DESI_TARGET']&desi_mask.ELG)==0))
print(np.count_nonzero(only_qso_not_elg))
targets_qso = targets[only_qso_not_elg]
ii_targets_qso = np.in1d(zcat['TARGETID'], targets_qso['TARGETID'])
print(np.count_nonzero(ii_targets_qso))
zcat_qso = zcat[ii_targets_qso]
print(set(zcat_qso['NUMOBS']))

395895
219986
{1, 2, 3, 4, 5, 6}


In [34]:
ii_targets = np.in1d(targets['TARGETID'], zcat_large['TARGETID'])
targets_large = targets[ii_targets]

In [39]:
names = []
for i in range(len(targets_large)):
    n = desi_mask.names(targets_large['DESI_TARGET'][i])
    if n not in names:
        names.append(n)

In [40]:
names

[['ELG', 'QSO', 'ELG_SOUTH', 'QSO_SOUTH'],
 ['LRG', 'LRG_1PASS', 'LRG_SOUTH', 'LRG_1PASS_SOUTH', 'BGS_ANY'],
 ['STD_WD', 'MWS_ANY'],
 ['QSO', 'QSO_SOUTH', 'STD_WD', 'BGS_ANY', 'MWS_ANY'],
 ['STD_FAINT', 'MWS_ANY'],
 ['STD_WD', 'BGS_ANY', 'MWS_ANY']]